# Sharmir's Secret Sharing

This notebook walks through how to add secret values under [Shamir's Secret Sharing](https://en.wikipedia.org/wiki/Shamir%27s_secret_sharing).

In [1]:
import time
import random
from math import ceil
from decimal import Decimal
from typing import List, Tuple
from IPython.display import HTML

import random
from sympy import Symbol, Poly
import pandas as pd

# Use a large prime as the modulus for arithmetic operations
# the prime must be larger than all secrets that will be added
prime: int = 180252380737439
threshold: int = 5
secret1: int = 123
secret2: int = 210

### Sample one random polynomial per secret

If using this as a base reference for another implementation and you are debugging, make `sample_shamir_polynomial` ***deterministic***!

`sample_shamir_polynomial` is the one source of randomness in Shamir's Secret Sharing algorithm, assuming the secrets and prime are fixed.

Example of making `sample_shamir_polynomial` deterministic:
```python
def sample_shamir_polynomial(zero_value: int) -> List[int]:
    coefs = [zero_value] + [prime-1 for _ in range(threshold - 1)]
    return coefs
```

All parties use the same prime and same threshold

In [16]:
'''
Generate a Shamir polynomial with a given zero_value.
'''
def sample_shamir_polynomial(zero_value: int) -> List[int]:
    coefs = [zero_value] + [random.randrange(prime) for _ in range(threshold - 1)]
    return coefs

In [3]:
'''
Evaluate the polynomial at a given point using the provided coefficients.
'''
def evaluate_at_point(coefs: List[int], point: int) -> int:
    result = 0
    for coef in reversed(coefs):
        result = (coef + point * result) % prime
    return result

In [4]:
'''
Perform polynomial interpolation at a given point using Lagrange interpolation.
'''
def interpolate_at_point(points_values: List[Tuple[int, int]], query_x_axis: int) -> int:
    x_vals, y_vals = zip(*points_values)
    constants = lagrange_constants_for_point(x_vals, query_x_axis)
    return sum(ci * vi for ci, vi in zip(constants, y_vals)) % prime

In [5]:
'''
Calculate Lagrange constants for the provided points and a target point.
'''
def lagrange_constants_for_point(points: List[int], query_x_axis: int) -> List[int]:
    constants = [0] * len(points)
    for i in range(len(points)):
        xi = points[i]
        num = 1
        denum = 1
        for j in range(len(points)):
            if j != i:
                xj = points[j]
                num = (num * (xj - query_x_axis)) % prime
                denum = (denum * (xj - xi)) % prime
        constants[i] = (num * pow(denum, -1, prime)) % prime
    return constants

In [6]:
'''
Split a secret into a list of shares.
'''
def shamir_share(secret: int, num_shares: int) -> List[Tuple[int, int]]:
    polynomial = sample_shamir_polynomial(secret)
    shares = [(i, evaluate_at_point(polynomial, i)) for i in range(1, num_shares + 1)]
    return shares, polynomial

In [7]:
'''
Reconstruct the secret from a list of shares.
'''
def shamir_reconstruct(shares: List[Tuple[int, int]], query_x_axis: int = 0) -> int:
    polynomial = [(p, v) for p, v in shares]
    secret = interpolate_at_point(polynomial, query_x_axis)
    return secret

### Secret shares

**Split `secret1` and `secret2` into secret shares.**

In [8]:
print(f'secret shares for {secret1}')
shares1, coeff1 = shamir_share(secret1, 10)
df = pd.DataFrame(shares1, columns=['x-value', 'y-value'])
HTML(df.to_html(index=False))

secret shares for 123


x-value,y-value
1,17882412767735
2,35676158534000
3,3284102599291
4,79252649270719
5,90014599649816
6,70646293844852
7,84615230233396
8,43527684724877
9,67633472235462
10,125321185213178


In [9]:
print(f'secret shares for {secret2}')
shares2, coeff2 = shamir_share(secret2, 10)
df = pd.DataFrame(shares2, columns=['x-value', 'y-value'])
HTML(df.to_html(index=False))

secret shares for 210


x-value,y-value
1,135579145190108
2,59457270332684
3,101821006798703
4,148928217998398
5,3612760856348
6,106294008761234
7,92957805666327
8,56923131251561
9,8084958711216
10,53166635491357


**To convince ourselves that the secret shares can be used to recover the secret, let's reconstruct the secret.**

In [10]:
print(f'reconstructed secret 1: {shamir_reconstruct(shares1, 0)}')
print(f'reconstructed secret 2: {shamir_reconstruct(shares2, 0)}')

reconstructed secret 1: 123
reconstructed secret 2: 210


### Add the secrets together under Shamir's

**Adding the secret shares together also adds the original secrets together**

In [11]:
def shamir_add(x, y):
    return [ (i+1, (xi[1] + yi[1]) % prime) for i, (xi, yi) in enumerate(list(zip(x, y))) ]

In [12]:
added = shamir_add(shares1, shares2)

### Coefficients of added polynomials under Shamir's

In [13]:
df = pd.DataFrame(added, columns=['x-value', 'y-value'])
HTML(df.to_html(index=False))

x-value,y-value
1,153461557957843
2,95133428866684
3,105105109397994
4,47928486531678
5,93627360506164
6,176940302606086
7,177573035899723
8,100450815976438
9,75718430946678
10,178487820704535


### Coefficients of added points under Shamir's

**Use these coefficients to reconstruct the secret which lies at x=0**

In [14]:
coefficients = [(i, shamir_reconstruct(added, i)) for i in range(len(added)+1)]
df = pd.DataFrame(coefficients, columns=['degree', 'coefficient'])
HTML(df.to_html(index=False))

degree,coefficient
0,333
1,153461557957843
2,95133428866684
3,105105109397994
4,47928486531678
5,93627360506164
6,176940302606086
7,177573035899723
8,100450815976438
9,75718430946678


### Recover the secret at x=0

In [15]:
print(f'secret: {shamir_reconstruct(added, 0)}')

secret: 333
